In [1]:
import requests
import json
from IPython.display import JSON
from keys import *

In [ ]:
#Be careful not to run too often as there is a maximum of 200 api calls per month

#Barcelona airport code LEBL
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/LEBL/2025-09-01T08:00/2025-09-01T20:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

In [ ]:
# Comprueba si la solicitud a la API fue exitosa (código de estado 200)
if response.status_code == 200:
    # Convierte la respuesta de la API a formato JSON (diccionario o lista de Python)
    new_data = response.json()

    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []
    
    # Agrega los nuevos datos descargados a la lista de datos existentes
    existing_data.append(new_data)

    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


In [ ]:
# Convierte la respuesta de la API a un diccionario o lista de Python
flight_arrivals = response.json()

# Accede a la lista de vuelos de llegada y extrae la información de cada vuelo
for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']
    flight_arrivals['arrivals'][0]['number']
    flight_arrivals['arrivals'][0]['arrival']['terminal']
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    flight_arrivals['arrivals'][0]['aircraft']['model']
    flight_arrivals['arrivals'][0]['airline']['name']

In [ ]:
# El código extrae datos de vuelos desde una respuesta JSON, construye un  DataFrame con información clave y concatena todos los registros para mostrar las primeras filas.

import pandas as pd
from numpy import nan

flight_arrivals = response.json()

def flight_extraction(flights):
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'],
        'flight_number': flights['number'],
        'from' : flights['departure']['airport']['name'],
        'airline' : flights['airline']['name'],
        'aircraft' : flights['aircraft']['model']
    })
    return(flights_df)

flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

flight_arrivals.head()

,flight_number,from,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,IB 5127,Palma De Mallorca,Iberia,Airbus A320,2025-09-01 06:20Z,2025-09-01 08:20+02:00
0,VY 3907,Palma De Mallorca,Vueling,Airbus A320,2025-09-01 06:20Z,2025-09-01 08:20+02:00
0,LA 5876,Palma De Mallorca,LATAM,Airbus A320,2025-09-01 06:20Z,2025-09-01 08:20+02:00
0,VY 6109,Rome,Vueling,Airbus A320 NEO,2025-09-01 06:15Z,2025-09-01 08:15+02:00
0,QR 3667,Rome,Qatar,Airbus A320 NEO,2025-09-01 06:15Z,2025-09-01 08:15+02:00
